In [1]:
#Change this for to create your folder and save experiments
#model_name = 'komal/gpt2-medium'

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

# Define the persistent path on your Drive
# IMPORTANT: This folder must exist in your Google Drive!
PERSISTENT_BASE_DIR = f'/content/drive/MyDrive/Efficient_AI_Project/EarlyExit_Experiments/{model_name}'
os.makedirs(PERSISTENT_BASE_DIR, exist_ok=True)

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import torch
import torch.nn.functional as F

from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import random
import itertools
import json

# -----------------------------
# Load datasets
# -----------------------------
def load_sst2():
    ds = load_dataset("glue", "sst2")
    return ds["train"], ds["validation"]

def load_rotten():
    ds = load_dataset("rotten_tomatoes")
    return ds["train"], ds["test"]

sst2_train, sst2_test = load_sst2()
rt_train, rt_test = load_rotten()


# -----------------------------
# Convert datasets to (text,label)
# -----------------------------
def convert_sst2(sample):
    return sample["sentence"], int(sample["label"])

def convert_rotten(sample):
    return sample["text"], int(sample["label"])

train_pairs = [convert_sst2(x) for x in sst2_train] + \
              [convert_rotten(x) for x in rt_train]

test_pairs  = [convert_sst2(x) for x in sst2_test] + \
              [convert_rotten(x) for x in rt_test]

# Optional shuffle
random.shuffle(train_pairs)
random.shuffle(test_pairs)

print(f"Total train samples: {len(train_pairs)}")
print(f"Total test samples: {len(test_pairs)}")


# -----------------------------
# Dataset class
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

class SentimentDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, label = self.data[idx]
        enc = self.tokenizer(
            text,
            add_special_tokens=False,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }


# -----------------------------
# Dynamic padding collate_fn
# -----------------------------
def collate_fn(batch):
    input_ids = [b["input_ids"] for b in batch]
    labels = torch.tensor([b["labels"] for b in batch])

    max_len = max(len(ids) for ids in input_ids)
    pad_id = tokenizer.pad_token_id

    padded = torch.full((len(batch), max_len), pad_id)
    for i, ids in enumerate(input_ids):
        padded[i, :len(ids)] = ids

    # attention mask: float
    attention_mask = (padded != tokenizer.eos_token_id).float()


    return {
        "input_ids": padded,
        "attention_mask": attention_mask,
        "labels": labels
    }

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Total train samples: 75879
Total test samples: 1938


In [3]:
# -----------------------------
# Create datasets
# -----------------------------
train_ds = SentimentDataset(train_pairs, tokenizer)
test_ds = SentimentDataset(test_pairs, tokenizer)

# -----------------------------
# Create dataloaders
# -----------------------------
train_loader = DataLoader(
    train_ds,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_ds,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)

print("Train loader batches:", len(train_loader))
print("Test loader batches:", len(test_loader))

# Optional: Inspect first batch
example = next(iter(train_loader))
print(example["input_ids"].shape)
print(example["attention_mask"].shape)
print(example["labels"])

Train loader batches: 4743
Test loader batches: 122
torch.Size([16, 39])
torch.Size([16, 39])
tensor([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [4]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer


class GPT2EarlyExitClassifier(nn.Module):
    def __init__(self, model_name, exit_layers, hyperparameters):
        super().__init__()

        # Load GPT-2 as causal LM (we will use only hidden states)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            output_hidden_states=True,   # <-- IMPORTANT
            return_dict=True
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.exit_layers = sorted(exit_layers)
        self.hp = hyperparameters
        self.num_labels = self.hp.get("num_labels", 2)
        dropout_rate = self.hp.get("dropout", 0.0)

        # Loss weights λ_e for each exit
        self.exit_loss_weights = self.hp.get(
            "exit_loss_weights",
            [1.0] * len(self.exit_layers)
        )

        hidden_size = self.model.config.hidden_size

        # Create classification heads for each exit layer
        self.exit_heads = nn.ModuleDict()
        for layer in self.exit_layers:
            self.exit_heads[str(layer)] = nn.Sequential(
                nn.Dropout(dropout_rate),
                nn.Linear(hidden_size, self.num_labels)
            )

        self.ce = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        Forward pass:
        - Calls GPT-2 normally (causal mask automatically handled)
        - Retrieves hidden_states for each layer
        - Applies classifier at each exit layer
        """

        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True
        )

        # hidden_states is a tuple:
        # [0] = embedding output
        # [1] = layer 1 output
        # ...
        # [12] = last layer output  (if GPT2 base)
        hidden_states = outputs.hidden_states

        logits_dict = {}
        total_loss = 0.0

        # For each early exit
        for i, layer in enumerate(self.exit_layers):

            # hidden_states[layer] has shape [batch, seq_len, hidden_dim]
            cls_vec = hidden_states[layer][:, -1, :]   # last token rep

            logits = self.exit_heads[str(layer)](cls_vec)
            logits_dict[layer] = logits

            # Add weighted loss
            if labels is not None:
                weight = self.exit_loss_weights[i]
                total_loss += weight * self.ce(logits, labels)

        return {
            "loss": total_loss if labels is not None else None,
            "logits": logits_dict
        }

In [5]:
hyperparameter_grid = {
    "num_labels": [2],                 # fixed
    "dropout": [0.0],
    "exit_loss_weights": [
        [1, 1, 1, 1, 1, 1, 1]],

    # training hyperparams
    "learning_rate": [1e-5],
    "weight_decay": [0.01],
    "num_epochs": [1],
    "max_grad_norm": [1.0],
    "batch_size": [16],

    # logging
    "log_every": [500]
}

exit_layers = [3, 6, 9, 12, 15, 18, 21]

In [6]:
def make_experiment_name(hp):
    return (
        f"lr{hp['learning_rate']}_"
        f"wd{hp['weight_decay']}_"
        f"ep{hp['num_epochs']}_"
        f"drop{hp['dropout']}"
    )

In [7]:
import torch.nn.functional as F
import numpy as np
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

# NOTE: The GPT2EarlyExitClassifier class definition must be available (as provided by you)

@torch.no_grad()
def early_exit_eval(model_path, hyperparameters, exit_layers, data_loader, device, thresholds):
    """
    Loads a trained GPT2EarlyExitClassifier and evaluates its efficiency
    (Accuracy vs. Average Exit Depth) across multiple confidence thresholds
    using batch processing.

    Args:
        model_path (str): Path to the saved model checkpoint (e.g., './checkpoints/exp_name/epoch_X').
        hyperparameters (dict): The original hyperparameters dictionary used for training.
        exit_layers (list): The list of exit layers used (e.g., [3, 6, 9, 12, 15, 18, 21]).
        data_loader (torch.utils.data.DataLoader): The evaluation data loader (e.g., test_loader).
        device (str): 'cuda' or 'cpu'.
        thresholds (list): List of confidence thresholds to test (e.g., [0.5, 0.7, 0.9]).

    Returns:
        dict: A dictionary containing metrics for each tested threshold.
    """

    print(f"\n--- Loading Model from: {model_path} ---")

    # 1. Instantiate the Model Architecture (Re-building the frame)
    model = GPT2EarlyExitClassifier(
        model_name="gpt2-medium",
        exit_layers=exit_layers,
        hyperparameters=hyperparameters
    ).to(device)

    # 2. Load the Model Weights (Filling the frame with trained weights)
    try:
        weights_path = os.path.join(model_path, 'pytorch_model.bin')
        state_dict = torch.load(weights_path, map_location=device)
        model.load_state_dict(state_dict)
        print("Model weights loaded successfully.")
    except Exception as e:
        print(f"FATAL ERROR: Could not load model weights from {weights_path}.")
        print(e)
        return {}

    model.eval()
    exit_layer_map = {layer: i + 1 for i, layer in enumerate(exit_layers)}
    num_exits = len(exit_layers)

    results = {}

    # Define how to calculate the total cost/depth: The average layer number * processed samples
    avg_cost_map = {layer: layer for layer in exit_layers}

    for th in thresholds:
        print(f"\nEvaluating Threshold: {th:.1f}")

        correct = 0
        total_samples = 0
        total_layers_used_sum = 0 # Sum of (exit_depth * samples_exited)

        start_time = time.time()

        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch["labels"]
            total_samples += labels.size(0)

            # 1. Run the forward pass to get all hidden states
            # We must use the model's inner transformer for efficient evaluation
            outputs = model.model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                output_hidden_states=True,
                return_dict=True
            )
            hidden_states = outputs.hidden_states

            # Track which samples in the batch have already exited
            batch_exited = torch.zeros(labels.size(0), dtype=torch.bool).to(device)

            # 2. Check each exit layer sequentially for early stopping
            for i, layer in enumerate(exit_layers):
                current_exit_depth = i + 1 # 1st, 2nd, 3rd exit...

                # Optimization: Stop loop if all samples have exited
                if batch_exited.all():
                    break

                # Retrieve last token vector (CLS representation)
                cls_vec = hidden_states[layer][:, -1, :]

                # Apply classification head
                logits = model.exit_heads[str(layer)](cls_vec)
                probabilities = F.softmax(logits, dim=-1)

                confidences, preds = torch.max(probabilities, dim=-1)

                # Identify samples that are ready to exit now
                # Condition: Not already exited AND confidence > threshold
                ready_to_exit = (~batch_exited) & (confidences >= th)

                # If this is the LAST layer, force all remaining samples to exit
                if current_exit_depth == num_exits:
                    ready_to_exit = ~batch_exited

                # 3. Update trackers for samples exiting in THIS step
                if ready_to_exit.any():
                    # Check accuracy for samples exiting NOW
                    correct_predictions = (preds == labels)[ready_to_exit]
                    correct += correct_predictions.sum().item()

                    # Accumulate cost: depth * number of exiting samples
                    total_layers_used_sum += (current_exit_depth * ready_to_exit.sum().item())

                    # Mark these samples as exited
                    batch_exited[ready_to_exit] = True

        end_time = time.time()

        # 4. Calculate Final Metrics
        inference_time = end_time - start_time

        final_accuracy = correct / total_samples if total_samples > 0 else 0
        avg_exit_depth = total_layers_used_sum / total_samples if total_samples > 0 else 0

        results[f'threshold_{th}'] = {
            "accuracy": final_accuracy,
            "avg_layers_used": avg_exit_depth,
            "avg_latency_sec": inference_time / total_samples if total_samples > 0 else 0,
            "cost_saving_pct": 100 * (1 - (avg_exit_depth / num_exits)),
            "tokens_per_sec": total_samples / inference_time if inference_time > 0 else 0 # Simplified token rate
        }

        print(f"  Accuracy: {results[f'threshold_{th}']['accuracy']:.4f}")
        print(f"  Avg Exit Depth (AED): {avg_exit_depth:.2f} / {num_exits}")
        print(f"  Cost Savings: {results[f'threshold_{th}']['cost_saving_pct']:.2f}%")

    return results

In [8]:
@torch.no_grad() # Disable gradient calculation for efficiency
def evaluate(model, data_loader, device):
    """
    Evaluates the Early Exit Classifier on a dataset.
    Reports average loss across all exits and (optional) basic accuracy.
    """
    model.eval() # Set model to evaluation mode (disables dropout, etc.)
    total_samples = 0
    total_loss = 0.0

    # Simple metric tracking for the final exit
    correct_predictions = 0

    print("--- Starting Evaluation ---")

    for step, batch in enumerate(data_loader):
        # 1. Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        labels = batch["labels"]
        total_samples += labels.size(0)

        # 2. Forward pass (returns loss and logits from all exits)
        out = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels # Pass labels to calculate the combined loss
        )

        # 3. Accumulate total loss
        total_loss += out["loss"].item()

        # 4. Get predictions from the LAST exit layer (typically the best)
        # Sort exit layers to ensure we always pick the highest/last index
        last_exit_layer = sorted(model.exit_heads.keys(), key=int)[-1]

        last_logits = out["logits"][int(last_exit_layer)]

        # Calculate predicted class (index with maximum logit)
        predictions = torch.argmax(last_logits, dim=-1)

        # 5. Accumulate correct predictions
        correct_predictions += (predictions == labels).sum().item()

    # 6. Calculate and print final metrics
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / total_samples

    print(f"\n--- Evaluation Results ---")
    print(f"Avg Combined Loss (All Exits): {avg_loss:.4f}")
    print(f"Accuracy (Final Exit Only): {accuracy:.4f}")
    print("--------------------------\n")

    # Return metrics if needed for tracking best model
    return avg_loss, accuracy

In [9]:
def train(model, train_loader, test_loader, hp, device):

    exp_name = make_experiment_name(hp)
    epoch_history = []

    # --- NEW: Identify the last exit layer here ---
    # This must be done inside the function since the model object is passed in
    last_exit_layer_key = sorted(model.exit_heads.keys(), key=int)[-1]
    last_exit_layer_int = int(last_exit_layer_key)
    # ---------------------------------------------

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=hp["learning_rate"],
        weight_decay=hp["weight_decay"]
    )

    for epoch in range(hp["num_epochs"]):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for step, batch in enumerate(train_loader, start=1):
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch["labels"]

            out = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=labels # Pass labels to calculate combined loss
            )

            loss = out["loss"]
            total_train_loss += loss.item()

            # --- NEW: Calculate Accuracy for the current batch (using final exit) ---
            total_train_samples += labels.size(0)

            # Get logits from the last exit head
            last_logits = out["logits"][last_exit_layer_int]
            predictions = torch.argmax(last_logits, dim=-1)
            total_train_correct += (predictions == labels).sum().item()
            # -----------------------------------------------------------------------

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), hp["max_grad_norm"])
            optimizer.step()

            if step % hp["log_every"] == 0:
                batch_accuracy = (predictions == labels).float().mean().item()
                print(f"Epoch {epoch+1} | Step {step} | Loss: {loss.item():.4f} | Acc: {batch_accuracy:.4f}")

        # --- Epoch End Metrics & Checkpointing ---
        avg_train_loss = total_train_loss / len(train_loader)
        avg_train_accuracy = total_train_correct / total_train_samples # CALCULATED HERE

        print(f"\n>>> Epoch {epoch+1} completed | Avg Train Loss: {avg_train_loss:.4f} | Avg Train Acc: {avg_train_accuracy:.4f}")

        # Test Metrics (Evaluation)
        final_test_loss, final_test_accuracy = evaluate(model, test_loader, device)

        # 1. Capture ALL epoch data in a dictionary
        epoch_metrics = {
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "train_accuracy": avg_train_accuracy,
            "test_loss": final_test_loss,
            "test_accuracy": final_test_accuracy
        }
        epoch_history.append(epoch_metrics)

        # 2. Save Checkpoint (with epoch number and dictionary)
        base_dir = "./checkpoints"
        exp_epoch_dir = os.path.join(base_dir, exp_name, f"epoch_{epoch+1}")

        os.makedirs(exp_epoch_dir, exist_ok=True)

        # A. Save model weights - 🛑 FIXED CODE HERE 🛑
        weights_path = os.path.join(exp_epoch_dir, 'pytorch_model.bin')

        # Save the state dictionary of your custom classifier
        torch.save(model.state_dict(), weights_path)

        # ----------------------------------------------------
        # The previous failed line: model.save_pretrained(exp_epoch_dir)
        # ----------------------------------------------------

        # B. Save the epoch metrics dictionary as a JSON file
        with open(os.path.join(exp_epoch_dir, 'metrics.json'), 'w') as f:
            json.dump(epoch_metrics, f, indent=4)

        print(f">>> Saved checkpoint and metrics to {exp_epoch_dir}")

    # Return the complete history (list of dictionaries)
    return epoch_history

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# generate all hyperparameter combinations
keys = list(hyperparameter_grid.keys())
values = list(hyperparameter_grid.values())

all_experiment_results = {}

for combo in itertools.product(*values):

    hp = dict(zip(keys, combo))
    hp["num_labels"] = 2  # fixed

    exp_name = make_experiment_name(hp)
    print("\n==============================")
    print("RUNNING EXPERIMENT:", exp_name)
    print("Hyperparameters:", hp)
    print("==============================")

    # 1. Create fresh model for this configuration
    model = GPT2EarlyExitClassifier(
        model_name="gpt2-medium",
        exit_layers=exit_layers,
        hyperparameters=hp
    ).to(device)

    # 2. Train
    experiment_history = train(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        hp=hp,
        device=device
    )

    # Inside the outer loop, you capture the history list and extract the final metrics
    final_metrics = experiment_history[-1]
    final_test_accuracy = final_metrics['test_accuracy'] # Key is 'test_accuracy'
    final_test_loss = final_metrics['test_loss']        # Key is 'test_loss'

    # You then STORE the simplified results using the following keys:
    all_experiment_results[exp_name] = {
        "hyperparameters": hp,
        "history": experiment_history,
        "final_test_accuracy": final_test_accuracy, # Stored as 'final_test_accuracy'
        "final_test_loss": final_test_loss
    }


RUNNING EXPERIMENT: lr1e-05_wd0.01_ep1_drop0.0
Hyperparameters: {'num_labels': 2, 'dropout': 0.0, 'exit_loss_weights': [1, 1, 1, 1, 1, 1, 1], 'learning_rate': 1e-05, 'weight_decay': 0.01, 'num_epochs': 1, 'max_grad_norm': 1.0, 'batch_size': 16, 'log_every': 500}


The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Epoch 1 | Step 500 | Loss: 3.1239 | Acc: 0.8750
Epoch 1 | Step 1000 | Loss: 1.7500 | Acc: 0.9375
Epoch 1 | Step 1500 | Loss: 1.3683 | Acc: 1.0000
Epoch 1 | Step 2000 | Loss: 3.7105 | Acc: 0.8750
Epoch 1 | Step 2500 | Loss: 1.0042 | Acc: 0.9375
Epoch 1 | Step 3000 | Loss: 1.2842 | Acc: 1.0000
Epoch 1 | Step 3500 | Loss: 2.9863 | Acc: 0.8750
Epoch 1 | Step 4000 | Loss: 0.6597 | Acc: 1.0000
Epoch 1 | Step 4500 | Loss: 0.4745 | Acc: 1.0000

>>> Epoch 1 completed | Avg Train Loss: 2.1614 | Avg Train Acc: 0.9039
--- Starting Evaluation ---

--- Evaluation Results ---
Avg Combined Loss (All Exits): 1.9902
Accuracy (Final Exit Only): 0.9236
--------------------------

>>> Saved checkpoint and metrics to ./checkpoints/lr1e-05_wd0.01_ep1_drop0.0/epoch_1


In [14]:
# --- Code to execute AFTER the hyperparameter search loop is complete ---
print("\n\n############################################")
print("HYPERPARAMETER SEARCH COMPLETE. FINDING BEST MODEL...")
print("############################################")

# 1. Identify the Best Model (based on final test accuracy)
try:
    best_exp_name = max(
        all_experiment_results,
        key=lambda name: all_experiment_results[name]["final_test_accuracy"]
    )

    best_result = all_experiment_results[best_exp_name]
    best_hp = best_result["hyperparameters"]
    best_history = best_result["history"]

    # Find the epoch with the highest accuracy from the saved history
    best_epoch_metrics = max(best_history, key=lambda x: x['test_accuracy'])
    best_epoch_number = best_epoch_metrics['epoch']

    # Construct the path to the best saved checkpoint
    # NOTE: Assuming base_dir = "./checkpoints" for the path structure
    BEST_CHECKPOINT_PATH = os.path.join(
        "./checkpoints",
        best_exp_name,
        f"epoch_{best_epoch_number}"
    )

    print(f"\n✅ BEST MODEL FOUND: {best_exp_name} (Epoch {best_epoch_number})")
    print(f"   Max Test Accuracy: {best_epoch_metrics['test_accuracy']:.4f}")
    print(f"   Checkpoint Path: {BEST_CHECKPOINT_PATH}")

except ValueError:
    print("Error: The results dictionary is empty. Cannot determine best model.")
    exit()

# 2. Run Early Exit Efficiency Evaluation
# Define the thresholds and exit layers for the final test
EVAL_THRESHOLDS = [0.5, 0.7, 0.8, 0.9]
# Note: exit_layers must be defined globally or loaded from best_hp if it varied.

print("\n--- Starting Final Early Exit Efficiency Evaluation ---")
final_efficiency_results = early_exit_eval(
    model_path=BEST_CHECKPOINT_PATH,
    hyperparameters=best_hp,
    exit_layers=exit_layers,
    data_loader=test_loader, # Use the global test_loader
    device=device,
    thresholds=EVAL_THRESHOLDS
)

# 3. Output Final Results
print("\n############################################")
print("FINAL EFFICIENCY RESULTS (Accuracy vs. Cost)")
print("############################################")
print(json.dumps(final_efficiency_results, indent=4))



############################################
HYPERPARAMETER SEARCH COMPLETE. FINDING BEST MODEL...
############################################

✅ BEST MODEL FOUND: lr1e-05_wd0.01_ep1_drop0.0 (Epoch 1)
   Max Test Accuracy: 0.9236
   Checkpoint Path: ./checkpoints/lr1e-05_wd0.01_ep1_drop0.0/epoch_1

--- Starting Final Early Exit Efficiency Evaluation ---

--- Loading Model from: ./checkpoints/lr1e-05_wd0.01_ep1_drop0.0/epoch_1 ---
Model weights loaded successfully.

Evaluating Threshold: 0.5
  Accuracy: 0.7890
  Avg Exit Depth (AED): 1.00 / 7
  Cost Savings: 85.71%

Evaluating Threshold: 0.7
  Accuracy: 0.8493
  Avg Exit Depth (AED): 1.35 / 7
  Cost Savings: 80.65%

Evaluating Threshold: 0.8
  Accuracy: 0.8782
  Avg Exit Depth (AED): 1.64 / 7
  Cost Savings: 76.52%

Evaluating Threshold: 0.9
  Accuracy: 0.9071
  Avg Exit Depth (AED): 2.26 / 7
  Cost Savings: 67.66%

############################################
FINAL EFFICIENCY RESULTS (Accuracy vs. Cost)
#############################